In [50]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [51]:
pbp = nfl.import_pbp_data([2024])
weekly_roster_data = nfl.import_weekly_rosters([2024])
roster_data = nfl.import_seasonal_rosters([2024])
earnings_data = pd.read_csv("overthecap/overthecap-earnings.csv")

weekly_roster_data.to_csv("weekly_roster_data.csv", index=False)

2024 done.
Downcasting floats.


In [ ]:
wr_data = weekly_roster_data[['team', 'week', 'player_name', 'player_id', 'position', 'status']]

wr_data_join_earnings = pd.merge(
    wr_data,
    earnings_data,
    left_on='player_name',
    right_on='player_name',
    how='left'
).sort_values(by=['team_x', 'week'])

# Fix: use .str.contains instead of .contains, and handle possible NaNs
wr_earnings_data = wr_data_join_earnings[["player_name", "position", "team_x", "team_y", "week", "status", "capnumber", "cash_paid"]]

wr_earnings_data[wr_earnings_data["player_name"] == "Aidan Hutchinson"]

,player_name,position,team_x,team_y,week,status,capnumber,cash_paid
25138,Aidan Hutchinson,DL,DET,Lions,1,ACT,9740015.0,3951672.0
25145,Aidan Hutchinson,DL,DET,Lions,2,ACT,9740015.0,3951672.0
25136,Aidan Hutchinson,DL,DET,Lions,3,ACT,9740015.0,3951672.0
25149,Aidan Hutchinson,DL,DET,Lions,4,ACT,9740015.0,3951672.0
25144,Aidan Hutchinson,DL,DET,Lions,6,ACT,9740015.0,3951672.0
25148,Aidan Hutchinson,LB,DET,Lions,7,RES,9740015.0,3951672.0
25151,Aidan Hutchinson,DL,DET,Lions,8,RES,9740015.0,3951672.0
25146,Aidan Hutchinson,DL,DET,Lions,9,RES,9740015.0,3951672.0
25143,Aidan Hutchinson,DL,DET,Lions,10,RES,9740015.0,3951672.0
25139,Aidan Hutchinson,DL,DET,Lions,11,RES,9740015.0,3951672.0


# Initial Analysis:
To get a general idea of spending into first downs, let's take the combined salaries of the offensive active roster for each team and divide by the number of successful plays in the 2024 season.

In [72]:
teams = wr_earnings_data["team_x"].unique()

active_wr_earnings_data = wr_earnings_data[wr_earnings_data["status"].str.contains("ACT", na=False)]

print(active_wr_earnings_data["position"].unique())

active_offense_earnings_data = active_wr_earnings_data[active_wr_earnings_data["position"].isin(["QB", "RB", "WR", "TE", "OL"])]

# Display rows where capnumber or cash_paid is NA
missing_salary_players = active_offense_earnings_data[
    active_offense_earnings_data["capnumber"].isna() | active_offense_earnings_data["cash_paid"].isna()
]["player_name"].drop_duplicates()

for name in missing_salary_players:
    print(name)
# for team in teams:
#     team_data = active_offense_earnings_data[active_offense_earnings_data["team_x"] == team]

#     players = team_data["player_name"].unique()

#     for player in players:
#         player_data = team_data[team_data["player_name"] == player]
#         print(player)
#         print(player_data[["week", "team_x", "position", "status", "capnumber", "cash_paid"]])
#         print("\n")

['K' 'LS' 'OL' 'WR' 'RB' 'DB' 'LB' 'DL' 'QB' 'P' 'TE']
Jon Gaines II
Carlos Washington
Patrick Ricard
Malaesala Aumavae-Laulu
Reggie Gilliam
Yosh Nijman
DJ Moore
Khari Blasingame
Nate Davis
Orlando Brown
Pierre Strong
Kadarius Toney
Mike Woods
KaVontae Turpin
TJ Bass
Michael Burton
Marvin Mims Jr.
Audric Estimé
Devaughn Vele
Dylan Leonard
Parker Hesse
Chris Brooks
Shaq Mason
Troy Hairston
Zach Thomas
Irv Smith
Michael Pittman
Gabe Davis
Josh Pederson
Jake Funk
Joshua Cephus
Joseph Fortson
Matthew Stafford
Joe Noteboom
Warren McClendon Jr.
DJ Chark
Laviska Shenault Jr.
DJ Turner
Thayer Munford Jr.
DJ Glaze
John Shenker
Robbie Chosen
Alec Ingold
Mathew Sexton
De'Von Achane
Gavin Hardison
Ireland Brown
Odell Beckham Jr.
Dee Eskridge
C.J. Ham
Mike Jordan
Mike Onwenu
Oli Udoh
Adam Prentice
Chris Reed
Jon Runyan
Jakob Johnson
Olumuyiwa Fashanu
Alec Lindstrom
Cam Jurgens
John Ross
Ben Skowronek
Rod Williams
Connor Williams
DK Metcalf
Olu Oluwatimi
TaMerik Williams
AJ Barner
Michael Jerrell
Ky

In [ ]:
def getInjuredPlayer(str):
    player_string_pattern = r"\d{1,2}-[A-Z].)"

find_injuries = pbp[pbp['desc'].str.lower().str.contains("injur")]

for index, play in find_injuries.iterrows():
    desc_sentences = play['desc'].split('. ')
    # Use list comprehension instead of .filter for lists
    print(play['play_id'])
    print(play['game_id'])
    print([s for s in desc_sentences if "injur" in s.lower()])
    print(desc_sentences)

199.0
2024_01_ARI_BUF
['BUF-7-Ta.Johnson was injured during the play']
['(11:08) (Shotgun) 1-K.Murray pass short middle to 4-G.Dortch to BUF 22 for 8 yards (43-T.Bernard, 9-T.Rapp)', 'BUF-7-Ta.Johnson was injured during the play', 'T.Johnson walks off.']
781.0
2024_01_ARI_BUF
['ARI-73-J.Williams was injured during the play']
['(:08) (Shotgun) 1-K.Murray pass short right to 31-E.Demercado to BUF 11 for 2 yards (31-R.Douglas)', 'ARI-73-J.Williams was injured during the play', 'J.Williams assisted off.']
2921.0
2024_01_ARI_BUF
['** Injury Update: ARI-74-I.Adams has returned to the game.']
['(13:35) 5-M.Prater 31 yard field goal is GOOD, Center-46-A.Brewer, Holder-12-B.Gillikin', '** Injury Update: ARI-74-I.Adams has returned to the game.']
2334.0
2024_01_BAL_KC
['BAL-53-K.Van Noy was injured during the play']
['(12:23) (Shotgun) 15-P.Mahomes pass incomplete short left to 9-J.Smith-Schuster', 'BAL-53-K.Van Noy was injured during the play', 'PENALTY on BAL-92-N.Madubuike, Roughing the Passe